In [2]:
import json
import pandas as pd
from PIL import Image
import PIL 
import numpy as np
import cv2
import matplotlib.pyplot as plt
import ast
import os

#### Update JSON path

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
# Required for Google collab
os.chdir('/content/drive/MyDrive/oil-tanker-project/notebooks')

In [5]:
#JSON Path
path_data = '../Oil Tanks/'

json_coco = json.load(open(path_data + 'labels_coco.json'))

In [6]:
#Functions to extract center and radius from JSON

def get_file(id):
    image_mapping ={x['id']:x['file_name'] for x in json_coco['images']}
    return image_mapping[id]

def get_bbox_center_radius(bbox):

    radius = int(0.5*max(bbox[2],bbox[3]))
    center = (int(bbox[0]+radius), int(bbox[1]+radius))

    return center, radius

#### JSON CSV path

In [7]:
csv_path =  '../data/b_box-dataframe/'

if not os.path.isdir(csv_path):
    os.mkdir(csv_path)

In [8]:
#Extracting B-Box data frame from JSON

annotations_mapping = {}
for x in json_coco['annotations']:
    # import ipdb; ipdb.set_trace()
    key=x['image_id']
    bbox=x['bbox']
    center, radius = get_bbox_center_radius(bbox)

    # if key is not in dict -> create list for key
    if key not in annotations_mapping:
        annotations_mapping[key] = [[],[],[]]
    
    l_centers_and_radius = annotations_mapping[key]
    l_centers_and_radius[0].append(bbox)
    l_centers_and_radius[1].append(center)
    l_centers_and_radius[2].append(radius)
    
df_annotation_mapping = pd.DataFrame([[key,value[0],value[1],value[2]] for key,value in annotations_mapping.items()])
df_annotation_mapping.columns = ['id','bbox', 'centers','radius']


df_annotation_mapping.to_csv(csv_path + 'data_preproc.csv')

In [9]:
#Preprocessing JSON B-Box data frame for Masking

df = pd.read_csv(csv_path + 'data_preproc.csv')

df['radius'] = df['radius'].apply(ast.literal_eval)
df['centers'] = df['centers'].apply(ast.literal_eval)

### update image patch 

In [10]:
img_path = '../Oil Tanks/image_patches/'

case1_path = '../data/case1_tanks/'
if not os.path.isdir(case1_path):
    os.mkdir(case1_path)

mask_path = '../data/case1_tanks/output_mask/'
if not os.path.isdir(mask_path):
    os.mkdir( mask_path)

In [11]:
# create masking from data_preproc file. "RUN ONLY IF REQUIRED" --> Time taking on google collab

for ix,data in df.iterrows():
        file_name = get_file(data['id'])
        
        image_cv = cv2.imread(img_path+file_name)

        mask = image_cv*0
                
        for idx in range(len(data['centers'])):
            r = data.radius[idx]
            c = list(data.centers[idx])

            mask = cv2.circle(mask, center= c, radius = r, color = (255, 255, 255), thickness = -1)
            
            #plt.imshow(mask)
        
        mask = mask[:,:,0] #extend dimension will be required for CNN model input 
        mask_image = Image.fromarray(mask)
        mask = mask_image.save(mask_path + file_name[:-4] + "_mask.jpg")
        #mask_arr.append(mask)
      

### update case-1 path (Input images with tanks only) 

In [12]:
case1_img_path = '../data/case1_tanks/input_image'

if not os.path.isdir(case1_img_path):
    os.mkdir(case1_img_path)

In [15]:
img_with_tank = [x.replace('_mask', '') for x in os.listdir(mask_path)]

In [16]:
for f in os.listdir(img_path):
    f_path = os.path.join(img_path, f)
    if f in img_with_tank:
        im = Image.open(f_path)
        im.save(case1_img_path + "/" + f)

In [13]:
# # extract input images with tanks

# for f_mask in os.listdir(mask_path):
#     f_mask_path = os.path.join(mask_path, f_mask)
#     for f in os.listdir(img_path):
#         f_path = os.path.join(img_path, f)
#         if f_mask_path.replace('_mask', '') == f_path:
#             im = Image.open(f_path)
#             save_image = im.save(case1_img_path + "/" + f)


In [14]:
# case1_img_path + "/"  + f

'../data/case1_tanks/input_image/27_8_8.jpg'